# Environment

In [2]:
import json
import re
import os

import pandas as pd

import psycopg2
from psycopg2 import Error

from tqdm.auto import tqdm

# Load project patches Data Frame
### A csv example file is available on the project repository

In [12]:
# Get project patches data and check number of total cases
df = pd.read_csv('patches_airflow.csv')
print('Total cases in project_patches.csv:')
print(len(df))

Total cases in project_patches.csv:
38478


# Connect to database and search for bugs and smells
### A database example file is available on the project repository

In [13]:
# Connect to an existing local database
connection = psycopg2.connect(host='127.0.0.1',
                              user='transfer_learning',
                              password='transfer_learning',
                              dbname='transfer_learning')
# Create a cursor to perform database operations
cursor = connection.cursor()
counter_success = 0

try:
    # Loop for all rows in the csv file
    for index, row in tqdm(df.iterrows()):
        project_csv = row['project']
        file_path_csv = row['file_path']
        file_name = os.path.splitext(os.path.basename(file_path_csv))[0]

        # Fetch classes if it's the same project name and file path as in the csv row
        postgreSQL_select_Query = 'SELECT * FROM public.class WHERE project = %s AND position(%s in file)>0'
        cursor.execute(postgreSQL_select_Query, (project_csv, file_name))
        classes = cursor.fetchall()

        # Loop for all classes in the database with the same project as the current csv row
        for case in tqdm(classes):

            # Set or reset bug fix flag as false for each case
            bug_fix_flag = False

            # Fetch methods if it's the same class id as the id of the class in case
            class_id = case[0]
            postgreSQL_select_Query = 'SELECT * FROM public.method WHERE class_id = %s'
            cursor.execute(postgreSQL_select_Query, (class_id,))
            methods = cursor.fetchall()

            # Findall to get only the numbers from all hunks, but separates in different tuples inside the list
            hunks = re.findall('@@ -(.*),(.*) [+](.*),(.*) @@', row['patch'])

            for hunk in hunks:
                # get line number from class and compare with hunk intervals
                class_line_number = case[7]
                hunk_start = int(hunk[0])
                hunk_end = int(hunk[0]) + int(hunk[1])

                # Check if line number in class table from database is within hunk intervals
                if hunk_start <= class_line_number <= hunk_end:
                    # Set bug fix flag as true if line number within intervals
                    bug_fix_flag = True

                for method in methods:
                    method_line_number = method[7]

                    # Check if line number in method table from database is also within hunk intervals
                    if hunk_start <= method_line_number <= hunk_end:
                        # Set bug fix flag as true if line number within intervals
                        bug_fix_flag = True

            # Create or reset default smells results with all false as default before testing
            smells_results = {
                'MultifacetedAbstraction': False,
                'UnnecessaryAbstraction': False,
                'InsufficientModularization': False,
                'WideHierarchy': False,
                'LongMethod': False,
                'ComplexMethod': False
            }

            # Get metrics for code smells for each class
            class_metrics = case[9]

            # TODO: Change verifying all smells to a single function if practical
            smell_LCOM = class_metrics.get('PercentLackOfCohesion', 0)
            smell_NOF = class_metrics.get('CountDeclClassVariable', 0) + class_metrics.get('CountDeclInstanceVariable', 0)
            smell_NOM = class_metrics.get('CountDeclMethod', 0)
            smell_NOPM = class_metrics.get('CountDeclMethodPublic', 0)
            smell_WMC = class_metrics.get('SumCyclomaticModified', 0)
            smell_NC = class_metrics.get('CountClassDerived', 0)

            # Individual test for each smell for each class
            if smell_LCOM >= 80 and smell_NOF >= 7 and smell_NOM >= 7:
                smells_results['MultifacetedAbstraction'] = True

            if smell_NOF >= 5 and smell_NOM == 0:
                smells_results['UnnecessaryAbstraction'] = True

            if smell_NOPM >= 20 or smell_NOM >= 30 or smell_WMC >= 100:
                smells_results['InsufficientModularization'] = True

            if smell_NC >= 10:
                smells_results['WideHierarchy'] = True

            for method in methods:
                # Get metrics for code smells for each method in that class
                method_metrics = method[9]

                smell_LOC = method_metrics.get('CountLine', 0)
                smell_CC = method_metrics.get('Cyclomatic', 0)

                # Individual test for each smell for each method
                if smell_LOC >= 100:
                    smells_results['LongMethod'] = True

                if smell_CC >= 8:
                    smells_results['ComplexMethod'] = True

            # Update class table marking as a bug fix commit
            postgreSQL_alter_Query = 'UPDATE public.class SET bug_fix = %s WHERE id = %s'
            cursor.execute(postgreSQL_alter_Query, (bug_fix_flag, class_id))
            connection.commit()

            # Update database with the test for smells results
            postgreSQL_alter_Query = 'UPDATE public.class SET smells = %s WHERE id = %s'
            cursor.execute(postgreSQL_alter_Query, (json.dumps(smells_results), class_id))
            connection.commit()

            # If it is a bug fix mark as a success
            if bug_fix_flag:
                counter_success += 1
            # # Conditional to check time needed and test script
            # if counter_success >= 1000:
                # raise Exception('Assigned counter limit reached')

except (Exception, Error) as error:
    print('Error while connecting to PostgreSQL', error)

finally:
    if connection:
        cursor.close()
        connection.close()
        print('PostgreSQL connection is closed')
        print('Successful cases:')
        print(counter_success)

0it [00:00, ?it/s]
100%|██████████| 4/4 [00:00<00:00,  6.10it/s]
0it [00:00, ?it/s]/it]
0it [00:00, ?it/s]/it]
100%|██████████| 1207/1207 [02:46<00:00,  7.26it/s]
0it [00:00, ?it/s]/it]
0it [00:00, ?it/s]/it]
0it [00:00, ?it/s]
0it [00:00, ?it/s]/it]
100%|██████████| 125/125 [00:36<00:00,  3.39it/s]
0it [00:00, ?it/s]s/it]
0it [00:00, ?it/s]s/it]
0it [00:00, ?it/s]
 12%|█▏        | 226/1922 [00:38<04:51,  5.82it/s]
17it [12:11, 43.02s/it] 


PostgreSQL connection is closed
Successful cases:
770


KeyboardInterrupt: 